In [1]:
import pandas as pd

# Load the dataset
valorant_data = pd.read_csv("datasets/valorant_model_dataset.csv")

print(valorant_data.head())

   Match Number  Round Number  Eco before purchase Drop Given to Teammate  \
0             1             1                  800                   none   
1             1             2                 3500                   none   
2             1             3                 3250                   none   
3             1             4                 4150                   none   
4             1             5                 3450                   none   

   Eco after purchase sidearms main weapon Shield type Friendly drop weapon  \
0                 100    ghost        none        none                 none   
1                  50    ghost    guardian       heavy                 none   
2                2250  classic        none       heavy             Guardian   
3                  50  classic      vandal       heavy                 none   
4                3050  classic      vandal       heavy                 none   

   Contingency Ability  Undercut Ability  kills round win/loss

In [2]:
from sklearn.preprocessing import LabelEncoder

# Step 2: Encode categorical columns
categorical_columns = ["sidearms", "main weapon", "Shield type", "Round Type", "Side"]

# Initialize label encoders
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    valorant_data[col] = le.fit_transform(valorant_data[col])
    label_encoders[col] = le  # Save encoders for later decoding

# Verify encoding
print(valorant_data[categorical_columns].head())

   sidearms  main weapon  Shield type  Round Type  Side
0         1            2            2           2     1
1         1            0            0           1     1
2         0            2            0           1     1
3         0            3            0           0     1
4         0            3            0           1     1


In [3]:
# Step 3: Define features and targets
X = valorant_data[[  # Features
    "Eco before purchase",
    "Contingency Ability",
    "Undercut Ability",
    "Total Utilities Used",
    "kills",
    "Round Type",
    "Side",
    "main weapon",  # Add current loadout
    "sidearms",
    "Shield type"
]]

y = valorant_data[[  # Targets
    "sidearms",
    "main weapon",
    "Shield type",
    "Contingency Ability",
    "Undercut Ability"
]]

# Verify shapes
print("Features Shape:", X.shape)
print("Targets Shape:", y.shape)

Features Shape: (66, 10)
Targets Shape: (66, 5)


In [4]:
from sklearn.model_selection import GroupKFold

# Step 4: Grouped K-Fold split
groups = valorant_data["Match Number"]
gkf = GroupKFold(n_splits=3)

# Split the data
for train_idx, test_idx in gkf.split(X, y, groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    break

# Verify shapes
print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)
print("Training Targets Shape:", y_train.shape)
print("Testing Targets Shape:", y_test.shape)

Training Features Shape: (42, 10)
Testing Features Shape: (24, 10)
Training Targets Shape: (42, 5)
Testing Targets Shape: (24, 5)


In [5]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

# Step 5: Train the model
# Initialize the base model
base_model = RandomForestClassifier(n_estimators=150, random_state=42)

# Wrap it with MultiOutputClassifier
multi_target_model = MultiOutputClassifier(base_model)

# Train the model
multi_target_model.fit(X_train, y_train)

print("Model training complete!")

Model training complete!


In [6]:
from sklearn.metrics import accuracy_score

# Step 6: Predict on the test set
y_pred = multi_target_model.predict(X_test)

# Evaluate accuracy for each target
accuracies = [
    accuracy_score(y_test.iloc[:, i], y_pred[:, i]) 
    for i in range(y_test.shape[1])
]

# Display accuracy for each target
target_columns = y_test.columns
for i, col in enumerate(target_columns):
    print(f"Accuracy for {col}: {accuracies[i]:.2f}")

# Calculate overall average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Overall Average Accuracy: {average_accuracy:.2f}")

Accuracy for sidearms: 0.88
Accuracy for main weapon: 0.83
Accuracy for Shield type: 0.88
Accuracy for Contingency Ability: 0.88
Accuracy for Undercut Ability: 0.71
Overall Average Accuracy: 0.83


In [7]:
# Example input for testing
player_input = pd.DataFrame([{
    "Eco before purchase": 2900,  # Player's credits
    "Contingency Ability": 0,     # Number of contingencies already owned
    "Undercut Ability": 1,        # Number of undercuts already owned
    "Total Utilities Used": 2,    # Example usage tracking
    "kills": 1,                   # Player performance
    "Round Type": 2,              # Encoded for "full buy"
    "Side": 1,                    # Encoded for "defenders"
    "main weapon": label_encoders["main weapon"].transform(["vandal"])[0],  # Current weapon
    "sidearms": label_encoders["sidearms"].transform(["classic"])[0],       # Current sidearm
    "Shield type": label_encoders["Shield type"].transform(["none"])[0]     # Current shield type
}])

# Predict recommendations
predictions = multi_target_model.predict(player_input)

# Decode predictions
sidearm_prediction = label_encoders["sidearms"].inverse_transform([predictions[0][0]])[0]
main_weapon_prediction = label_encoders["main weapon"].inverse_transform([predictions[0][1]])[0]
shield_prediction = label_encoders["Shield type"].inverse_transform([predictions[0][2]])[0]
contingency_prediction = predictions[0][3]
undercut_prediction = predictions[0][4]

# Display recommendations
print("Recommended Loadout:")
print(f"Sidearm: {sidearm_prediction}")
print(f"Main Weapon: {main_weapon_prediction}")
print(f"Shield: {shield_prediction}")
print(f"Contingency Ability: {'Yes' if contingency_prediction == 1 else 'No'}")
print(f"Undercut Ability: {'Yes' if undercut_prediction == 1 else 'No'}")

Recommended Loadout:
Sidearm: classic
Main Weapon: none
Shield: none
Contingency Ability: No
Undercut Ability: Yes


In [8]:
def adjust_recommendations(economy, sidearm, main_weapon, shield, contingency, undercut):
    costs = {
        "sidearms": {"sheriff": 800, "classic": 0, "ghost": 500, "none": 0},
        "main weapon": {"vandal": 2900, "bulldog": 2050, "spectre": 1600, "none": 0},
        "shield": {"light": 400, "heavy": 1000, "none": 0},
        "abilities": {"contingency": 200, "undercut": 200}
    }

    # Prioritize main weapon
    if main_weapon == "none" and economy >= 2900:
        main_weapon = "vandal"
        economy -= costs["main weapon"]["vandal"]
    elif main_weapon == "none" and economy >= 2050:
        main_weapon = "bulldog"
        economy -= costs["main weapon"]["bulldog"]

    # Prioritize shield upgrade
    if shield == "none" and economy >= 1000:
        shield = "heavy"
        economy -= costs["shield"]["heavy"]
    elif shield == "none" and economy >= 400:
        shield = "light"
        economy -= costs["shield"]["light"]

    # Prioritize utilities
    if economy >= 200 and undercut < 2:
        undercut += 1
        economy -= costs["abilities"]["undercut"]

    if economy >= 200 and contingency == 0:
        contingency = 1
        economy -= costs["abilities"]["contingency"]

    return sidearm, main_weapon, shield, contingency, undercut, economy

In [9]:
def predict_and_refine(player_input):
    # Predict recommendations
    predictions = multi_target_model.predict(player_input)

    # Decode predictions
    sidearm_prediction = label_encoders["sidearms"].inverse_transform([predictions[0][0]])[0]
    main_weapon_prediction = label_encoders["main weapon"].inverse_transform([predictions[0][1]])[0]
    shield_prediction = label_encoders["Shield type"].inverse_transform([predictions[0][2]])[0]
    contingency_prediction = predictions[0][3]
    undercut_prediction = predictions[0][4]

    # Extract input economy
    economy = player_input["Eco before purchase"].iloc[0]

    # Refine recommendations
    refined_sidearm, refined_main_weapon, refined_shield, refined_contingency, refined_undercut, remaining_eco = adjust_recommendations(
        economy=economy,
        sidearm=sidearm_prediction,
        main_weapon=main_weapon_prediction,
        shield=shield_prediction,
        contingency=contingency_prediction,
        undercut=undercut_prediction
    )

    # Output recommendations
    print("Final Recommendations:")
    print(f"Sidearm: {refined_sidearm}")
    print(f"Main Weapon: {refined_main_weapon}")
    print(f"Shield: {refined_shield}")
    print(f"Contingency Ability: {'Yes' if refined_contingency == 1 else 'No'}")
    print(f"Undercut Ability: {refined_undercut}")
    print(f"Remaining Economy: {remaining_eco}")

In [10]:
player_input = pd.DataFrame([{
    "Eco before purchase": 4900,
    "Contingency Ability": 0,
    "Undercut Ability": 1,
    "Total Utilities Used": 2,
    "kills": 2,
    "Round Type": 2,
    "Side": 1,
    "main weapon": label_encoders["main weapon"].transform(["none"])[0],
    "sidearms": label_encoders["sidearms"].transform(["classic"])[0],
    "Shield type": label_encoders["Shield type"].transform(["none"])[0]
}])

predict_and_refine(player_input)

Final Recommendations:
Sidearm: classic
Main Weapon: vandal
Shield: heavy
Contingency Ability: Yes
Undercut Ability: 2
Remaining Economy: 600
